Dosyaların okunması

In [ ]:
# logoları crop edip dosya dosya ayıracak

In [ ]:
# 1
# data klasörünü oku (train için kullandığın dosya)
# resim isimlerini al os.listdir()
# logo_uuid.png # böl
# logoya göre böl # dict içerisine logo: ["logo_uuid.png", "logo_uuid.png", "logo_uuid.png"]

# !!json var ise oku, json içerisindeki logoları ele!!

# resim okuman grecek
# modele sor
# modelden çıkanları cropla bir listeye at
# crop edilen alanın boyutuna
# 20 tane logon varsa hepsinin boyutuna sahipsin -> 30x40 32x43, bu boyutların hepsinin ortalamasını alırsın
# sonra resimleri bu ortalama göre resize edersin!
# aldığın ortalamayı da bir json içerisine kaydedersin

# api -> akbank -> size: 80,20 -> dışarıdan akbank fotoğrafı geldi resize 80,20 -> feature matching yap




# 
# klasörü de sen yaratıcaksın kodla ve sınıf isminda olacak
# onlarıda sırayla indexine göre logo adı ile beraber akbank_0 akbank_1 olarak akbank klasörüne kaydet


In [ ]:
#bu kod train ve validdeki dosyaları tek bir dosyaya, images ve labels olarak topluyor

import os 
import shutil
def move_files(src_folder,dst_folder):

    # Get a list of all files in the source folder
    files = os.listdir(src_folder)

    # Iterate over the list of files
    for file in files:
        # Construct the full path of the file in the source folder
        src_path = os.path.join(src_folder, file)
        # Construct the full path of the file in the destination folder
        dst_path = os.path.join(dst_folder, file)
        # Check if the file is a regular file
        if os.path.isfile(src_path):
            # Move the file to the destination folder
            shutil.copy2(src_path, dst_path)

In [ ]:
move_files("dataset/train/images/","dataset/whole/images/")
move_files("dataset/valid/images/","dataset/whole/images/")

move_files("dataset/train/labels/","dataset/whole/labels/")
move_files("dataset/valid/labels/","dataset/whole/labels/")

In [ ]:


import cv2
import yaml
from itertools import chain

def crop_logos():
    image_dict = {}
    with open("dataset/data.yaml", 'r') as file:
        # load the contents of the file into a Python object
        data = yaml.load(file, Loader=yaml.FullLoader)

    # extract the names from the data
    class_yaml = data["names"]

    img_folder_path1 = "dataset/train/images/"
    img_folder_path2 = "dataset/valid/images/"

    txt_folder_path1 = "dataset/train/labels/"
    txt_folder_path2 = "dataset/valid/labels/"
    
    cropped_folder_path = "cropped2/"
    
    class_names = class_yaml
    counter = 0

    # loop through all the images in the image folder
    for img_name in chain(os.listdir(img_folder_path1),os.listdir(img_folder_path2)):
        

        # construct the full path to the image and txt file
        img_path = os.path.join(img_folder_path, img_name)
        txt_path = os.path.join(txt_folder_path, img_name.replace('.jpg', '.txt'))
        
        # read the image and the txt file
        img = cv2.imread(img_path)

        #get image shapes to convert txt coordinates into pixel
        height, width = img.shape[0], img.shape[1]
        
        try:
            with open(txt_path) as f:
                lines = f.read().strip().split('\n')
            
            for line in lines:
                coordinate_list = list()
                counter+=1
                if counter %100==0:
                    print(counter, "images cropped")
                # split the line by space to separate class name and coordinates
                #txt file has information like that 12 0.123 0.323 0.110 0.05 == class name, x,y,w,h
                class_name, x, y, w, h = line.strip().split()
                
                label_name = class_names[int(class_name)]
                x, y, w, h = float(x), float(y), float(w), float(h)
                #x1, y1, x2, y2 = int((x-w/2) * width), int((y-h/2) * height), int((x+w/2) * width), int((y+h/2) * height)
                x1, y1, x2, y2 = round((x-w/2) * width), round((y-h/2) * height), round((x+w/2) * width), round((y+h/2) * height)
                #print(x1,y1,x2,y2)
                # crop the image using the coordinates
                cropped_img = img[y1:y2, x1:x2]

                # construct the full path to the save the cropped image
                cropped_img_path = os.path.join(cropped_folder_path, f"{label_name}_{counter}.jpg")
    
                if label_name in image_dict:
                    coordinate_list = image_dict[label_name]
                #coordinate_list.append([f"{label_name}_{counter}.jpg",x1, y1, x2, y2])
                coordinate_list.append([f"{label_name}_{counter}.jpg",x2-x1,y2-y1])

                image_dict.update({label_name:coordinate_list})
                
                # save the cropped image
                cv2.imwrite(cropped_img_path, cropped_img)
                
        except ValueError:
            print('File is empty. Skipping...')
            print("problem with file",txt_path )
            continue
    return image_dict



In [ ]:
cropped_dict  = crop_logos()

In [ ]:
cropped_dict

In [ ]:
cropped_dict["trendyol"]

In [ ]:
"""from itertools import islice
avg_dict = dict()
#for label_name in islice(cropped_dict,3):
for label_name in cropped_dict:
    x1_total, y1_total, x2_total, y2_total = 0, 0, 0, 0
    div = len(cropped_dict[label_name])
    for element in cropped_dict[label_name]:
        x1_total += element[1]
        y1_total += element[2]
        x2_total += element[3]
        y2_total += element[4]
    
    x1_avg, y1_avg, x2_avg, y2_avg = round(x1_total/div), round(y1_total/div), round(x2_total/div), round(y2_total/div)
    avg_dict.update({label_name:[x1_avg, y1_avg, x2_avg, y2_avg]})   
   
"""
        

In [ ]:
from itertools import islice
avg_dict = dict()
#for label_name in islice(cropped_dict,3):
for label_name in cropped_dict:
    x_total, y_total = 0, 0
    div = len(cropped_dict[label_name])
    for element in cropped_dict[label_name]:
        x_total += element[1]
        y_total += element[2]

    
    x_avg, y_avg = round(x_total/div), round(y_total/div)
    avg_dict.update({label_name:[x_avg, y_avg]})   
   

        

In [ ]:
avg_dict

In [ ]:
folder_path = "cropped2/" 
resized_path = "reshaped/"
counter = 0
for label_name in cropped_dict:
    for element in cropped_dict[label_name]:

        avg_size = tuple(avg_dict[label_name])
        img_path = os.path.join(folder_path,element[0])
        
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, avg_size)
        cv2.imwrite(resized_path+element[0], img_resized)
        counter+=1
        if counter%100 ==0:
            print(counter)

In [ ]:
avg_dict["trendyol"]

In [ ]:
img = cv2.imread("C:\Users\halil\Desktop\feature-matching\a101-logo_1.jpg")
img_resized = cv2.resize(img, avg_size)
cv2.imwrite(resized_path+element[0], img)